# Desafio 1 - Maratona Behind the Code 2021

## Bibliotecas

In [ ]:
import os
from datetime import timedelta
from pathlib import Path
from timeit import default_timer as timer

import matplotlib.pyplot as plt
import pandas as pd
import requests
import seaborn as sns
from dotenv import find_dotenv, load_dotenv
from ibm_watson_machine_learning import APIClient
from IPython.display import HTML, display
from sklearn import set_config
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import (AdaBoostClassifier, BaggingClassifier,
                              ExtraTreesClassifier, RandomForestClassifier,
                              StackingClassifier)
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, plot_confusion_matrix
from sklearn.model_selection import GridSearchCV  # , RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier

## Configurações

In [ ]:
seed: int = 0
data_path: Path = Path("../desafio/assets/data/")
set_config(display="diagram")

## Dados

### Junção dos datasets

In [ ]:
accounts = pd.read_csv(data_path / "ACCOUNTS.csv", index_col="ID")
account_cols = list(accounts.columns)
print(account_cols)

In [ ]:
demographics = pd.read_csv(data_path / "DEMOGRAPHICS.csv", index_col="ID")
demographic_cols = list(demographics.columns)
print(demographic_cols)

In [ ]:
loans = pd.read_csv(data_path / "LOANS.csv", index_col="ID")
loan_cols = list(loans.columns)
print(loan_cols)

In [ ]:
answers = pd.read_csv(data_path / "ANSWERS.csv")
df_all = pd.concat([accounts, demographics, loans], axis=1).reset_index()[answers.columns]

### Informações gerais

In [ ]:
df_all.info()

- A maioria das variáveis (colunas) são numéricas, mas há algumas categóricas.
- Com exceção da variável que identifica cada cliente (`ID`) e da variável de destino (`ALLOW`), todas têm dados faltantes (nulos).

### Divisão dos dados entre treino e teste

Antes de inspecionar os dados, serão reservados alguns exemplos para teste, que não serão vistos durante a análise ou modelagem dos dados:

In [ ]:
test_size = 500
target = "ALLOW"
df_train, df_test = train_test_split(df_all, test_size=test_size, random_state=seed, stratify=df_all[[target]])
print(f'Dimensões dos dados de treino: {df_train.shape}')
print(f'Dimensões dos dados de teste: {df_test.shape}')

### Variável destino

A variável destino para este desafio é a `ALLOW`, significando se um empréstimo deverá ser permitido ou não, baseado nas informações dadas. Vamos dar uma olhada em como está a distribuição dessa variável

In [ ]:
risk_plot = sns.countplot(
    data=df_train, x=target, order=df_train[target].value_counts().index
)
plt.show()

### Dados de exemplo

In [ ]:
with pd.option_context("display.max_columns", None):
    display(df_train.sample(10, random_state=seed))

### Variáveis categóricas

In [ ]:
non_numeric_cols = list(df_train.select_dtypes(exclude='number').columns)

In [ ]:
with pd.option_context("display.max_columns", None):
    display(df_train.describe(include="O"))

A quantidade de valores únicos para as variáveis `CHECKING_BALANCE` e `EXISTING_SAVINGS` é relativamente grande (>30%), e observando-se os dados de exemplo, há indícios de que boa parte dos valores possam ser numéricos:

In [ ]:
max_unique_treshold = 0.3
cols_with_many_unique_values = df_train[non_numeric_cols].nunique() > max_unique_treshold * len(df_train)
possibly_numeric_cols = list(df_train[non_numeric_cols].columns[cols_with_many_unique_values])
possibly_numeric_cols

In [ ]:
for col in possibly_numeric_cols:
    display(
        df_train[col]
        .value_counts()
        .reset_index()
        .sort_values([col, "index"], ascending=False)
        .rename(columns={col: "Quantidade", "index": "Valor"})
        .set_index("Valor")
        .head(3)
    )

Na hora da modelagem, teremos algumas opções do que fazer com estas colunas:
- Tratar essas colunas como variáveis numéricas, deixando que valores textuais como `NO_CHECKING` e `UNKNOWN` sejam substituídos por `NaN` (e posteriormente imputar um valor, por exemplo, zero).
- Se for relevante distinguir o caso anterior dos zeros que já estavam nestas colunas, poderia ser criada uma nova variável binária cujo valor fosse `True` quando a variável original fosse não numérica, e `False` nos demais casos

### Remoção de texto em variáveis numéricas

Por hora, apenas para visualizar estatísticas sobre os valores numéricos, valores textuais nessas colunas serão trocados por `NaN` usando um *transformer* personalizado:

#### Instalação de um pacote extra

In [ ]:
!rm -rf custom_sklearn_transformers
!git clone git@github.com:he7d3r/custom_sklearn_transformers.git
!git --git-dir custom_sklearn_transformers/.git archive -o custom_sklearn_transformers.zip HEAD
!pip install --quiet --upgrade pip custom_sklearn_transformers.zip

#### Uso do pacote

In [ ]:
from custom_sklearn_transformers.transformers import ToNumeric

In [ ]:
to_numeric = ToNumeric(possibly_numeric_cols, errors='coerce')

Para criar um modelo capaz de fazer transformações nos dados de entrada, vamos criar uma `Pipeline` do `scikit-learn` e aplicar nossas transformações de pré-processamento dentro dos estágios dela.

In [ ]:
preprocessor = Pipeline([
    ('to_numeric', to_numeric),
])
clean_df = preprocessor.fit_transform(df_train)

Estes são os valores que as outras variáveis não numéricas assumem:

In [ ]:
sorted_cat_cols = list(sorted(set(non_numeric_cols) - set(possibly_numeric_cols)))
for col in sorted_cat_cols:
    display(clean_df[col].value_counts().to_frame().T)

- Todas as variáveis poderiam ser convertidas em valores numéricos usando "One-Hot-Encoding".
- Alternativamente, algumas variáveis poderiam poderia ser convertidas para valores crescentes em uma ordem que aparenta ir da pior para a melhor situação (mas esta é uma avaliação subjetiva):
  - **`CREDIT_HISTORY`**: `PRIOR_PAYMENTS_DELAYED`, `NO_CREDITS`, `CREDITS_PAID_TO_DATE`, `ALL_CREDITS_PAID_BACK`, `OUTSTANDING_CREDIT`
  - **`OTHERS_ON_LOAN`**: `NONE`, `CO-APPLICANT`, `GUARANTOR`
  - **`HOUSING`**: `FREE`, `RENT`, `OWN`

### Variáveis numéricas

In [ ]:
with pd.option_context("display.max_columns", None):
    display(clean_df.describe(exclude="O"))

Sinta-se livre para ver a distribuição de outras colunas do conjunto de dados, utilizar os outros conjuntos de dados, explorar as correlações entre variáveis e outros.

In [ ]:
numeric_cols = list(clean_df.select_dtypes(include='number').columns)

In [ ]:
with pd.option_context('display.max_columns', 6):
    for col in numeric_cols:
        display(clean_df[col].value_counts().to_frame().T)

### Correlações

#### Variáveis sobre as contas

In [ ]:
sorted_account_cols = list(sorted(set(account_cols + [target])))
sns.pairplot(clean_df[sorted_account_cols], hue=target, palette='Set1', corner=True)
plt.show()

#### Variáveis sobre dados demográficos

In [ ]:
sorted_demographic_cols = list(sorted(set(demographic_cols + [target])))
sns.pairplot(clean_df[sorted_demographic_cols], hue=target, palette='Set1', corner=True)
plt.show()

- Talvez seja melhor considerar que o tipo de trabalho (`JOB_TYPE`) é uma variável categórica e fazer One-Hot-Encoding, assumindo que não exista necessariamente uma ordenação natural dos tipos

#### Variáveis sobre os empéstimos

In [ ]:
sorted_loan_cols = list(sorted(set(loan_cols + [target])))
sns.pairplot(clean_df[sorted_loan_cols], hue=target, palette='Set1', corner=True)
plt.show()

### Variáveis categóricas (gráficos)

In [ ]:
for col in sorted_cat_cols:
    risk_plot = sns.countplot(
        data=df_train, y=col, order=df_train[col].value_counts().index, orient='h', hue='ALLOW', palette='Set1'
    )
    plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=len(sorted_cat_cols), sharex=True, figsize=(10,20))
for i, col in enumerate(sorted_cat_cols):
    group_means = df_train.groupby([col])['ALLOW'].mean().rename('ALLOW_RATE').sort_values(ascending=False).to_frame()
    sns.barplot(x='ALLOW_RATE', y=group_means.index, data=group_means, ax=axes[i])

## Tratamento dos dados

Uma vez que exploramos os dados, entendemos a importância de cada coluna e podemos fazer alterações nelas para para obter um melhor resultado.

#### Dados de entrada

O desafio espera um modelo que aceite todas as variáveis dos conjuntos de dados disponíveis (exceto a variável destino, `ALLOW`):

In [ ]:
challenge_columns = [
    "ID",
    "CHECKING_BALANCE",
    "PAYMENT_TERM",
    "CREDIT_HISTORY",
    "LOAN_PURPOSE",
    "LOAN_AMOUNT",
    "EXISTING_SAVINGS",
    "EMPLOYMENT_DURATION",
    "INSTALLMENT_PERCENT",
    "SEX",
    "OTHERS_ON_LOAN",
    "CURRENT_RESIDENCE_DURATION",
    "PROPERTY",
    "AGE",
    "INSTALLMENT_PLANS",
    "HOUSING",
    "EXISTING_CREDITS_COUNT",
    "JOB_TYPE",
    "DEPENDENTS",
    "TELEPHONE",
    "FOREIGN_WORKER",
]

#### Variáveis categóricas

Observando a execução do método `.info()` acima, podemos ver que existem colunas do tipo `object`. O modelo do `scikit-learn` que vamos usar não é capaz de processar uma variável desse tipo. Portanto, para dar seguimento ao experimento, será utilizada a técnica de _one-hot encoding_ para tratamento de variáveis categóricas. Além disso, a coluna `ID` será desconsiderada, pois sabemos que ela não é uma informação útil para a predição (é apenas um número identificando um cliente).

Primeiramente, especificaremos quais variáveis serão tratadas como categóricas e quais serão tratadas como numéricas:

In [ ]:
categorical_features = [
    "CREDIT_HISTORY",
    "LOAN_PURPOSE",
    "SEX",
    "OTHERS_ON_LOAN",
    "PROPERTY",
    "INSTALLMENT_PLANS",
    "HOUSING",
    "TELEPHONE",  # Boolean  # TODO: Impute some value instead OneHotEncoding the missing values
    "FOREIGN_WORKER",  # Boolean  # TODO: Impute some value instead OneHotEncoding the missing values
]
numeric_features = [
    "CHECKING_BALANCE",
    "PAYMENT_TERM",
    "LOAN_AMOUNT",
    "EXISTING_SAVINGS",
    "EMPLOYMENT_DURATION",
    "INSTALLMENT_PERCENT",
    "CURRENT_RESIDENCE_DURATION",
    "AGE",
    "EXISTING_CREDITS_COUNT",
    "JOB_TYPE", # NOTE: This could also be considered categorical
    "DEPENDENTS",
]

Qualquer outra variável será desconsiderada:

In [ ]:
features = numeric_features + categorical_features
unwanted_columns = list(
    sorted((set(challenge_columns) - set([target])) - set(features))
)
print(unwanted_columns)

A pipeline de pré-processamento será atualizada para que realize transformações específicas para cada tipo de variável:

In [ ]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
numeric_transformer = Pipeline(steps=[
    ('to_numeric', to_numeric),
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
    ]
)
# display(preprocessor)

Nesta nova versão:
- As variáveis `CHECKING_BALANCE` e `EXISTING_SAVINGS` continuarão sendo convertidas para tipos numéricos
- One-Hot-Encoding será aplicado às variáveis categóricas, resultando em uma coluna para cada valor

A título de exemplo, a antiga coluna `SEX` foi transformada em novas colunas, uma para cada valor:

In [ ]:
preprocessor.fit(df_train)
# onehot_features = list(preprocessor.named_transformers_["cat"]["onehot"].get_feature_names_out())
onehot_features = list(preprocessor.named_transformers_["cat"]["onehot"].get_feature_names())  # scikit-learn=0.23
clean_df = pd.DataFrame(
    preprocessor.transform(df_train),
    columns=numeric_features + onehot_features,
    index=df_train.index
)
# clean_df.head(4)
clean_df[clean_df.columns[clean_df.columns.str.endswith('_F') | clean_df.columns.str.endswith('_M')]].head(4)

Esta é a lista completa das colunas criadas pelo One-Hot-Encoding:

In [ ]:
print(onehot_features)

#### Variáveis faltantes

Com as etapas de pré-processamento definidas até aqui, algumas colunas ainda têm valores faltantes:

In [ ]:
clean_df[clean_df.columns[clean_df.isnull().sum() > 0]].info()

Nestes casos, faremos apenas um tratamento simples, de imputar o valor zero nas linhas que tiverem faltando algum valor. Não necessariamente essa técnica é a melhor para se utilizar no desafio, é apenas um exemplo de como tratar o dataset.

Tratamentos mais avançados, como modificação de colunas ou criação de novas colunas, serão incluídos na `Pipeline` posteriormente, se necessário.

In [ ]:
imputer = SimpleImputer(strategy='constant', fill_value=0)

numeric_transformer = Pipeline(steps=[
    ('to_numeric', to_numeric),
    ('imputer', imputer),
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
    ]
)
# display(preprocessor)

In [ ]:
preprocessor.fit(df_train)
clean_df = pd.DataFrame(
    preprocessor.transform(df_train),
    columns=numeric_features + onehot_features,
    index=df_train.index
)
# display(clean_df.head(2))
# clean_df.info()

A nova pipeline de pré-processamento zera o número de colunas com dados faltantes:

In [ ]:
clean_df.isnull().sum().sum()

#### Escalamento de variáveis contínuas

Para evitar problemas com modelos sensíveis a diferentes escalas numéricas, vamos normalizar as variáveis numéricas para que tenham média zero e desvio padrão unitário. A título de exemplo, antes de normalizar os dados, estas são as variáveis com maior e menor desvio padrão, respectivamente:

In [ ]:
top_std = clean_df.std().sort_values(ascending=False).rename('Desvio padrão').to_frame()
display(pd.concat([top_std.head(1), top_std.tail(1)]))

In [ ]:
imputer = SimpleImputer(strategy='constant', fill_value=0)

numeric_transformer = Pipeline(steps=[
    ('to_numeric', to_numeric),
    ('imputer', imputer),
    ('scaler', StandardScaler()),
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
    ]
)
# display(preprocessor)

preprocessor.fit(df_train)
clean_df = pd.DataFrame(
    preprocessor.transform(df_train),
    columns=numeric_features + onehot_features,
    index=df_train.index
)
# display(clean_df.head(2))
# clean_df.info()

Agora, o desvio padrão está próximo de um:

In [ ]:
top_std_normalized = clean_df.std()[top_std.index].rename('Desvio padrão').to_frame()
display(pd.concat([top_std_normalized.head(1), top_std_normalized.tail(1)]))

## Criação do modelo

Com os dados prontos, podemos selecionar um modelo de Machine Learning para treinar com nossos dados. Nesse exemplo, vamos utilizar um modelo de classificação básico, o de Árvore de Decisão.

In [ ]:
classifier = DecisionTreeClassifier(random_state=seed)
pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('clf', classifier)
])

Pronto! Essa pipeline agora está pronta para receber todas as variáveis do desafio, transformá-las e passar para o modelo aquelas que forem relevantes:

In [ ]:
# display(pipe)

Abaixo, separamos os dados que queremos predizer dos dados que utilizamos como informações para a predição.

In [ ]:
X_train = df_train[challenge_columns]
y_train = df_train[target]
X_test = df_test[challenge_columns]
y_test = df_test[target]
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
param_grid = [
    {
    'clf':[LinearSVC(random_state=seed)],
    'clf__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'clf__penalty': ['l1', 'l2'],
    },
    {
    'clf': [GaussianNB()],
    },
    {
    'clf': [DecisionTreeClassifier(random_state=seed)],
    'clf__max_depth': [3, 5, 7, 9, 11, 13, 15],
    'clf__max_features': [0.5, 0.7, 0.9],
    'clf__min_samples_split': [5, 10, 15],
    'clf__min_samples_leaf': [2, 4, 8],
    'clf__criterion': ['gini', 'entropy'],
    },
    {
    'clf': [KNeighborsClassifier()],
    'clf__n_neighbors': [5, 10, 20, 40, 80],
    'clf__weights': ['uniform', 'distance'],
    },
    {
    'clf': [LogisticRegression(solver='liblinear', random_state=seed)],
    'clf__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'clf__penalty': ['l1', 'l2'],
    },
    {
    'clf': [ExtraTreesClassifier(n_estimators=10, random_state=seed)],
    'clf__n_estimators': [50, 75, 100, 150, 200],
    },
    {
    'clf': [AdaBoostClassifier(n_estimators=10, random_state=seed)],
    'clf__n_estimators': [10, 50, 100],
    'clf__learning_rate': [0.5, 1.0],
    },
    {
    'clf':[SVC(random_state=seed)],
    'clf__C': [0.01, 0.1, 1, 10, 100],
    'clf__gamma': [0.01, 0.1, 1, 10, 100],
    },
    {
    'clf': [RandomForestClassifier(n_estimators=10, random_state=seed)],
    'clf__max_depth': [5, 10],
    'clf__n_estimators': [50, 100],
    'clf__max_features': [0.4, 0.8],
    'clf__min_samples_split': [2, 10],
    'clf__min_samples_leaf': [1, 10],
    'clf__criterion': ['gini', 'entropy'],
    },
    {
    'clf': [BaggingClassifier(n_estimators=5, random_state=seed)],
    'clf__base_estimator': [SVC(C=0.3, gamma=0.2, random_state=seed)],
    'clf__n_estimators': [5, 10],
    'clf__max_samples': [0.7, 0.9],
    'clf__max_features': [0.4, 0.8],
    },
    {
    'clf': [MLPClassifier(max_iter=10000, random_state=seed)],
    'clf__alpha': [0.0001, 0.001, 0.01],
    'clf__hidden_layer_sizes': [50, 75, 100],
    'clf__learning_rate_init': [0.0001, 0.001],
    },
    # {
    # 'clf': [StackingClassifier(estimators=[
    #     ('lsvc', LinearSVC(C=0.007, penalty='l2', random_state=seed)),
    #     ('gnb', GaussianNB()),
    #     ('dtc', DecisionTreeClassifier(max_depth=7, max_features=0.8, min_samples_split=11, min_samples_leaf=2, criterion='gini', random_state=seed)),
    #     ('knn', KNeighborsClassifier(n_neighbors=65, weights='uniform')),
    #     ('lr', LogisticRegression(C=5, penalty='l2', solver='liblinear', random_state=seed)),
    #     ('etc', ExtraTreesClassifier(n_estimators=93, random_state=seed)),
    #     ('abc', AdaBoostClassifier(n_estimators=81, learning_rate=0.4, random_state=seed)),
    #     ('svc', SVC(C=0.3, gamma=0.2, random_state=seed)),
    #     ('rfc', RandomForestClassifier(n_estimators=99, max_depth=30, max_features=0.35, min_samples_split=32, min_samples_leaf=4, criterion='gini', random_state=seed)),
    #     ('mlpc', MLPClassifier(max_iter=10000, alpha=0.25, hidden_layer_sizes=225, learning_rate_init=0.0002, random_state=seed)),
    # ])],
    # 'clf__final_estimator': [SVC()]
    # },
]

In [ ]:
cv_num = 2
verbose = 1
scoring = ['f1', 'accuracy', 'precision', 'recall']

cv_train_size = int(((cv_num - 1) / cv_num) * len(X_train))
cv_test_size = len(X_train) - cv_train_size


# cv = RandomizedSearchCV(pipe, param_distributions=param_grid, n_jobs=-1, verbose=verbose, random_state=seed,
#                           cv=cv_num, n_iter=15, scoring=scoring)
cv = GridSearchCV(pipe, param_grid, cv=cv_num, n_jobs=-1, verbose=verbose, scoring=scoring, refit=scoring[0])

start = timer()
cv.fit(X_train, y_train)
end = timer()
print(f'Tempo gasto para treinar o modelo {cv_num} vezes com cada combinação de parâmetros para encontrar a melhor: {timedelta(seconds=end-start)}')

In [ ]:
display(HTML('<h4>Dados brutos da validação cruzada</h4>'))
print(f'Cada combinação de parâmetros foi utilizada em {cv_num} rodadas de treino com {cv_train_size} exemplos e teste com {cv_test_size}.')

rank_col = 'rank_test_' + scoring[0]
cv_df = pd.DataFrame(cv.cv_results_).sort_values(rank_col).set_index(rank_col)

cols = ['mean_test_f1', 'mean_test_accuracy', 'mean_test_precision', 'mean_test_recall', 'mean_fit_time', 'params']
with pd.option_context('display.max_colwidth', None):
    display(cv_df[cols].head(20))

In [ ]:
temp = pd.read_pickle('cv_df_10x12_12min.zip')
print(temp.sort_values('mean_fit_time')[['mean_fit_time', 'param_clf']])

In [ ]:
model = cv.best_estimator_

display(HTML(f'<h4>Maior média de {scoring[0]}</h4>'))
display(HTML(f'''<ul>
<li>Maior média de {scoring[0]}: {cv.best_score_:.5f} (desvio padrão: {cv.cv_results_["std_test_" + scoring[0]][cv.best_index_]:.5f})</li>
<li>Parâmetros que maximizaram a média de {scoring[0]}:<br/><code>{cv.best_params_}</code></li>
<li>Modelo com a maior média de {scoring[0]}:</li>
</ul>'''))
display(model)

In [ ]:
y_pred = model.predict(X_test)
score = f1_score(y_test, y_pred)
print(
    f"F1-score do modelo no conjunto de testes: {score:.5f}"
)

In [ ]:
# cmd = ConfusionMatrixDisplay.from_predictions(y_test, y_pred, labels=[0, 1], display_labels=['Negar', 'Aprovar'])
cmd = plot_confusion_matrix(model, X_test, y_test, display_labels=['Negar', 'Aprovar'])  # scikit-learn=0.23
cmd.ax_.set(xlabel='Previsão', ylabel='Realidade')
plt.show()

## Watson Machine Learning

As etapas da publicação serão colocadas em funções individuais.

Primeiro, será usando um cliente da API do Watson Machine Learning para definir o espaço de publicação padrão:

#### Funções auxiliares

In [ ]:
def get_client(API_KEY):
    location = 'us-south'
    wml_credentials = {
        "apikey": API_KEY,
        "url": 'https://' + location + '.ml.cloud.ibm.com'
    }
    return APIClient(wml_credentials)

def set_default_space(client):
    # The DEPLOYMENT_SPACE_GUID was copied from the output of
    # client.spaces.list(limit=10)
    DEPLOYMENT_SPACE_GUID = os.getenv("DEPLOYMENT_SPACE_GUID")
    client.set.default_space(DEPLOYMENT_SPACE_GUID)

O transformador customizado será enviado para o WML:

In [ ]:
def store_user_package_extension(client):
    meta_prop_pkg_extn = {
        client.package_extensions.ConfigurationMetaNames.NAME: "Custom_Sklearn_Transformers",
        client.package_extensions.ConfigurationMetaNames.DESCRIPTION: "Extensão para transformações personalizadas",
        client.package_extensions.ConfigurationMetaNames.TYPE: "pip_zip"
    }

    # Subir o pacote
    pkg_extn_details = client.package_extensions.store(meta_props=meta_prop_pkg_extn,
                                                       file_path="custom_sklearn_transformers.zip")

    # Salvar as informações sobre o pacote
    pkg_extn_uid = client.package_extensions.get_uid(pkg_extn_details)
    return pkg_extn_uid

Vamos agora criar uma especificação de software com o nosso pacote customizado, para que o WML possa utilizar.

In [ ]:
def store_software_specification(client, pkg_extn_uid):
    base_sw_spec_uid = client.software_specifications.get_uid_by_name("default_py3.8")
    
    # Metadados da nova especificação de software
    meta_prop_sw_spec = {
        client.software_specifications.ConfigurationMetaNames.NAME: "sw_spec_custom_sklearn_transformers",
        client.software_specifications.ConfigurationMetaNames.DESCRIPTION: "Especificação de software com transformações personalizadas",
        client.software_specifications.ConfigurationMetaNames.BASE_SOFTWARE_SPECIFICATION: {"guid": base_sw_spec_uid}
    }

    # Criando a nova especificação de software e obtendo seu ID
    sw_spec_details = client.software_specifications.store(meta_props=meta_prop_sw_spec)
    sw_spec_uid = client.software_specifications.get_uid(sw_spec_details)

    # Adicionando o pacote customizado à nova especificação
    client.software_specifications.add_package_extension(sw_spec_uid, pkg_extn_uid)
    return sw_spec_uid

Finalmente, vamos publicar a pipeline utilizando a especificação de software customizada que criamos.

In [ ]:
def store_model(client, sw_spec_uid, model):
    # Metadados do modelo
    model_props = {
        client.repository.ModelMetaNames.NAME: "Pipeline customizada",
        client.repository.ModelMetaNames.TYPE: 'scikit-learn_0.23',
        client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sw_spec_uid
    }

    # Publicando a Pipeline como um modelo
    # This creates a new Model asset in the deployment space on Watson Machine Learning:
    # https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=cpdaas
    published_model = client.repository.store_model(model=model, meta_props=model_props)
    published_model_uid = client.repository.get_model_uid(published_model)
    client.repository.get_details(published_model_uid)
    return published_model_uid

Agora que o modelo está salvo, vamos deixá-lo disponível online, para que possamos testá-lo:

In [ ]:
def create_deploy(client, published_model_uid):
    # Metadados para publicação do modelo
    metadata = {
        client.deployments.ConfigurationMetaNames.NAME: "Publicação do modelo customizado",
        client.deployments.ConfigurationMetaNames.ONLINE: {}
    }

    # Publicar
    created_deployment = client.deployments.create(published_model_uid, meta_props=metadata)

    # There should be a new Deployment on the space:
    # https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=cpdaas
    return created_deployment

Adaptando um trecho de código fornecido na referência da API, na página de deployment do modelo, podemos testar a API implementada:

In [ ]:
def make_test_request_to_api(client, deployment_id):
    meta_props = {
        "input_data": [
            {
                "fields": [
                    "ID",
                    "CHECKING_BALANCE", "PAYMENT_TERM", "CREDIT_HISTORY", "LOAN_PURPOSE",
                    "LOAN_AMOUNT", "EXISTING_SAVINGS", "EMPLOYMENT_DURATION", "INSTALLMENT_PERCENT",
                    "SEX", "OTHERS_ON_LOAN", "CURRENT_RESIDENCE_DURATION", "PROPERTY",
                    "AGE", "INSTALLMENT_PLANS", "HOUSING", "EXISTING_CREDITS_COUNT",
                    "JOB_TYPE", "DEPENDENTS", "TELEPHONE", "FOREIGN_WORKER",
                ],
                "values": [
                    [
                        1234,
                        None, 987.0, None, "CAR_NEW",
                        4567.0, None, 5.0, 4.0,
                        "M", "NONE", 3.0, "SAVINGS_INSURANCE",
                        42.0, "NONE", "OWN", None,
                        3.0, 1.0, 1.0, 1.0,
                    ],
                    [
                        4321,
                        "NO_CHECKING", 333.0, "PRIOR_PAYMENTS_DELAYED", "BUSINESS",
                        3333.3, 111.1, 13.0, 3.0,
                        "F", "BANK", 2.0, "REAL_ESTATE",
                        33.0, "NONE", "RENT", 1.0,
                        2.0, 2.0, 0.0, 0.0,
                    ],
                ],
            }
        ]
    }
    response_scoring = client.deployments.score(deployment_id, meta_props)

    # See documentation at https://cloud.ibm.com/apidocs/machine-learning?code=python#deployments-compute-predictions
    print("Scoring response")
    display(response_scoring)

#### Publicação do modelo e teste pela API

Quando estiver pronto para publicar o modelo, basta descomentar as linhas da célula abaixo:

In [ ]:
# load_dotenv(find_dotenv())
# API_KEY = os.getenv("API_KEY")

# client = get_client(API_KEY)
# set_default_space(client)
# pkg_extn_uid = store_user_package_extension(client)
# sw_spec_uid = store_software_specification(client, pkg_extn_uid)
# published_model_uid = store_model(client, sw_spec_uid, pipe)
# created_deployment = create_deploy(client, published_model_uid)
# deployment_uid = client.deployments.get_uid(created_deployment)
# make_test_request_to_api(client, deployment_uid)

# print("O modelo está publicado! "
#       "Para submeter o desafio, basta acessar https://maratona.dev/challenge/1, "
#       "e utilizar as credenciais abaixo para realizar a submissão:")
# print("Credenciais para envio (não compartilhe esses dados com ninguém!)\n\n"
#       f"API key: {API_KEY}\nDeployment ID: {deployment_uid}")

## Preenchimento do arquivo de respostas

In [ ]:
answers['ALLOW'] = model.predict(answers)
answers.to_csv("ANSWERS.csv", index=False)

with pd.option_context('display.max_columns', 6):
    display(answers.sample(2, random_state=seed))

In [ ]:
answers['ALLOW'].value_counts(normalize=True).to_frame().T

In [ ]:
!rm -rf código.zip
!zip --exclude '*/.git/*' -rq código.zip custom_sklearn_transformers notebook.ipynb
!ls código.zip